In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely import wkt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
import rasterio
from tqdm import tqdm
from joblib import Parallel, delayed
import os
from datetime import datetime

In [2]:
data = pd.read_csv('data.csv')

In [3]:
data.head(3)

,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,...,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry,dataset,tif_path
0,1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,25-11-2023,14-04-2024,97,54,...,Transplanting,Flood,Groundwater,4,87,17,Rabi,POLYGON ((78.18079255482755 17.978971746424413...,train,/kaggle/input/sentineltimeseriesdata/SentinelT...
1,1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,13-11-2023,26-04-2024,82,58,...,Transplanting,Flood,Canal,5,94,15,Rabi,POLYGON ((78.17483419891283 17.981508840879556...,train,/kaggle/input/sentineltimeseriesdata/SentinelT...
2,1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,19-12-2023,28-04-2024,92,91,...,Transplanting,Flood,Canal,3,99,20,Rabi,POLYGON ((78.16888476438905 17.976727511659835...,train,/kaggle/input/sentineltimeseriesdata/SentinelT...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11791 entries, 0 to 11790
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   FarmID           11791 non-null  int64 
 1   category         8775 non-null   object
 2   Crop             11791 non-null  object
 3   State            11791 non-null  object
 4   District         11791 non-null  object
 5   Sub-District     11791 non-null  object
 6   SDate            11791 non-null  object
 7   HDate            11791 non-null  object
 8   CropCoveredArea  11791 non-null  int64 
 9   CHeight          11791 non-null  int64 
 10  CNext            11791 non-null  object
 11  CLast            11791 non-null  object
 12  CTransp          11791 non-null  object
 13  IrriType         11791 non-null  object
 14  IrriSource       11791 non-null  object
 15  IrriCount        11791 non-null  int64 
 16  WaterCov         11791 non-null  int64 
 17  ExpYield         11791 non-null

In [5]:
data = data.dropna(subset=['tif_path'])

### **EXTRACTING FEATURES FROM *tif_path***

In [6]:
# Define the local directory where the TIFF files are stored
local_tif_directory = 'C:/Users/Aduragbemi/Documents/TCHC/tif_file/'

# Update the 'tif_path' column to use local paths
data['tif_path'] = data['tif_path'].apply(lambda x: os.path.join(local_tif_directory, os.path.basename(x)))

# Check the updated paths
print(data['tif_path'].head())

0    C:/Users/Aduragbemi/Documents/TCHC/tif_file/20...
1    C:/Users/Aduragbemi/Documents/TCHC/tif_file/20...
2    C:/Users/Aduragbemi/Documents/TCHC/tif_file/20...
3    C:/Users/Aduragbemi/Documents/TCHC/tif_file/20...
4    C:/Users/Aduragbemi/Documents/TCHC/tif_file/20...
Name: tif_path, dtype: object


In [7]:
# Example feature calculation functions
def calculate_ndvi(nir_band, red_band):
    """Calculate NDVI (Normalized Difference Vegetation Index)."""
    ndvi = (nir_band - red_band) / (nir_band + red_band)
    return np.nanmean(ndvi)

def calculate_evi(nir_band, red_band, blue_band):
    """Calculate EVI (Enhanced Vegetation Index)."""
    evi = 2.5 * (nir_band - red_band) / (nir_band + 6 * red_band - 7.5 * blue_band + 1)
    return np.nanmean(evi)

def calculate_ndwi(nir_band, green_band):
    """Calculate NDWI (Normalized Difference Water Index)."""
    ndwi = (green_band - nir_band) / (green_band + nir_band)
    return np.nanmean(ndwi)

def calculate_gndvi(nir_band, green_band):
    """Calculate GNDVI (Green Normalized Difference Vegetation Index)."""
    gndvi = (nir_band - green_band) / (nir_band + green_band)
    return np.nanmean(gndvi)

def calculate_savi(nir_band, red_band, L=0.5):
    """Calculate SAVI (Soil Adjusted Vegetation Index)."""
    savi = ((nir_band - red_band) / (nir_band + red_band + L)) * (1 + L)
    return np.nanmean(savi)

def calculate_msavi(nir_band, red_band):
    """Calculate MSAVI (Modified Soil Adjusted Vegetation Index)."""
    msavi = (2 * nir_band + 1 - np.sqrt((2 * nir_band + 1)**2 - 8 * (nir_band - red_band))) / 2
    return np.nanmean(msavi)

def calculate_area(src, nir_band):
    """Calculate the area of farmland using the NIR band and pixel resolution."""
    # Get the transform (affine transformation matrix)
    transform = src.transform
    # Get the CRS (coordinate reference system)
    crs = src.crs
    # Get the pixel resolution (in meters or degrees)
    pixel_size_x = transform.a  # Width of a pixel in the x-direction
    pixel_size_y = -transform.e  # Height of a pixel in the y-direction (negative due to rasterio's coordinate system)

    # Calculate the area of a single pixel
    if crs.is_geographic:  # CRS is in degrees (e.g., WGS84)
        bounds = src.bounds
        center_lat = (bounds.top + bounds.bottom) / 2
        pixel_size_x_meters = pixel_size_x * 111320 * np.cos(np.radians(center_lat))
        pixel_size_y_meters = pixel_size_y * 111320
        pixel_area = pixel_size_x_meters * pixel_size_y_meters
    else:  # CRS is in meters (e.g., UTM)
        pixel_area = pixel_size_x * pixel_size_y

    # Create a mask for farmland (e.g., NDVI > 0.3)
    farmland_mask = nir_band > 0.3  # Adjust the threshold as needed

    # Count the number of farmland pixels
    farmland_pixels = np.sum(farmland_mask)

    # Calculate the total area of the farmland
    total_area = farmland_pixels * pixel_area

    return total_area


In [8]:
# Function to calculate features for each row in the data
def process_row_for_features(index, row):
    features = {'index': index}

    # Retrieve the TIFF file path and check if it's valid
    tif_path = row['tif_path']
    if not isinstance(tif_path, str):
        print(f"Skipping entry due to missing tif_path for index {index}")
        # Add NaN for all features if path is missing
        features.update({
            'ndvi': np.nan,
            'evi': np.nan,
            'ndwi': np.nan,
            'gndvi': np.nan,
            'savi': np.nan,
            'msavi': np.nan,
            'area': np.nan
        })
        return features

    # Open the TIFF file and read bands for feature calculation
    with rasterio.open(tif_path) as src:
        red = src.read(3)    # B4 for Red
        green = src.read(2)  # B3 for Green
        blue = src.read(1)   # B2 for Blue
        nir = src.read(4)    # B8 for NIR

        # Calculate each feature
        features['ndvi'] = calculate_ndvi(nir, red)
        features['evi'] = calculate_evi(nir, red, blue)
        features['ndwi'] = calculate_ndwi(nir, green)
        features['gndvi'] = calculate_gndvi(nir, green)
        features['savi'] = calculate_savi(nir, red)
        features['msavi'] = calculate_msavi(nir, red)
        features['area'] = calculate_area(src, nir)

    return features

In [9]:
new_features = Parallel(n_jobs=-1)(delayed(process_row_for_features)(index, row)
                                   for index, row in tqdm(data.iterrows(), total=len(data)))

100%|██████████| 10595/10595 [01:05<00:00, 161.07it/s]


In [10]:
new_features_df = pd.DataFrame(new_features).set_index('index')
data = data.join(new_features_df)

data.head(3)

,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,...,geometry,dataset,tif_path,ndvi,evi,ndwi,gndvi,savi,msavi,area
0,1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,25-11-2023,14-04-2024,97,54,...,POLYGON ((78.18079255482755 17.978971746424413...,train,C:/Users/Aduragbemi/Documents/TCHC/tif_file/20...,0.100756,-0.793684,8.488031,0.127153,0.151125,4232.596191,27108.609824
1,1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,13-11-2023,26-04-2024,82,58,...,POLYGON ((78.17483419891283 17.981508840879556...,train,C:/Users/Aduragbemi/Documents/TCHC/tif_file/20...,0.188090,0.564248,11.493762,0.187815,0.282110,3249.392822,24730.359593
2,1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,19-12-2023,28-04-2024,92,91,...,POLYGON ((78.16888476438905 17.976727511659835...,train,C:/Users/Aduragbemi/Documents/TCHC/tif_file/20...,0.206596,-1.456745,10.166884,0.206553,0.309869,3741.956055,15694.697856


### **CONVERT AREA TO HECTARES**

In [11]:
data['AreaHectares'] = data['area'] / 10000

data.head(3)

,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,...,dataset,tif_path,ndvi,evi,ndwi,gndvi,savi,msavi,area,AreaHectares
0,1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,25-11-2023,14-04-2024,97,54,...,train,C:/Users/Aduragbemi/Documents/TCHC/tif_file/20...,0.100756,-0.793684,8.488031,0.127153,0.151125,4232.596191,27108.609824,2.710861
1,1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,13-11-2023,26-04-2024,82,58,...,train,C:/Users/Aduragbemi/Documents/TCHC/tif_file/20...,0.188090,0.564248,11.493762,0.187815,0.282110,3249.392822,24730.359593,2.473036
2,1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,19-12-2023,28-04-2024,92,91,...,train,C:/Users/Aduragbemi/Documents/TCHC/tif_file/20...,0.206596,-1.456745,10.166884,0.206553,0.309869,3741.956055,15694.697856,1.569470


### **CONVERT *SDate* AND *HDate* FEATURE TO DATETIME**

In [12]:
data['SDate'] = pd.to_datetime(data['SDate'], dayfirst=True)
data['HDate'] = pd.to_datetime(data['HDate'], dayfirst=True)

print(data['SDate'].dtype)
print(data['HDate'].dtype)

datetime64[ns]
datetime64[ns]


### **EXTRACT *month* AND *week_of_month* FROM *SDate* AND *HDate***

In [13]:
def week_of_month(dt):
    """Calculate the week of the month for a given datetime object."""
    first_day = dt.replace(day=1)
    dom = dt.day  # Day of the month
    adjusted_dom = dom + first_day.weekday()  # Adjust for the first week's weekday
    return (adjusted_dom - 1) // 7 + 1

In [14]:
data['SMonth'] = data['SDate'].dt.month
data['SWOM'] = data['SDate'].apply(week_of_month)

data['HMonth'] = data['HDate'].dt.month
data['HWOM'] = data['HDate'].apply(week_of_month)

data.head(3)

,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,...,ndwi,gndvi,savi,msavi,area,AreaHectares,SMonth,SWOM,HMonth,HWOM
0,1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25,2024-04-14,97,54,...,8.488031,0.127153,0.151125,4232.596191,27108.609824,2.710861,11,4,4,2
1,1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13,2024-04-26,82,58,...,11.493762,0.187815,0.282110,3249.392822,24730.359593,2.473036,11,3,4,4
2,1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19,2024-04-28,92,91,...,10.166884,0.206553,0.309869,3741.956055,15694.697856,1.569470,12,4,4,4


### **GROWING SEASON DURATION**

In [15]:
data['SeasonDuration'] = (data['HDate'] - data['SDate']).dt.days

data.head(3)

,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,...,gndvi,savi,msavi,area,AreaHectares,SMonth,SWOM,HMonth,HWOM,SeasonDuration
0,1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25,2024-04-14,97,54,...,0.127153,0.151125,4232.596191,27108.609824,2.710861,11,4,4,2,141
1,1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13,2024-04-26,82,58,...,0.187815,0.282110,3249.392822,24730.359593,2.473036,11,3,4,4,165
2,1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19,2024-04-28,92,91,...,0.206553,0.309869,3741.956055,15694.697856,1.569470,12,4,4,4,131


In [16]:
data.columns

Index(['FarmID', 'category', 'Crop', 'State', 'District', 'Sub-District',
       'SDate', 'HDate', 'CropCoveredArea', 'CHeight', 'CNext', 'CLast',
       'CTransp', 'IrriType', 'IrriSource', 'IrriCount', 'WaterCov',
       'ExpYield', 'Season', 'geometry', 'dataset', 'tif_path', 'ndvi', 'evi',
       'ndwi', 'gndvi', 'savi', 'msavi', 'area', 'AreaHectares', 'SMonth',
       'SWOM', 'HMonth', 'HWOM', 'SeasonDuration'],
      dtype='object')

### **DROP FEATURES**

In [17]:
data = data.drop(['State', 'geometry', 'tif_path', 'area', 'SDate', 'HDate'], axis=1)
data.head(3)

,FarmID,category,Crop,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,...,ndwi,gndvi,savi,msavi,AreaHectares,SMonth,SWOM,HMonth,HWOM,SeasonDuration
0,1326576,Healthy,Paddy,Medak,Kulcharam,97,54,Pea,Lentil,Transplanting,...,8.488031,0.127153,0.151125,4232.596191,2.710861,11,4,4,2,141
1,1326577,Healthy,Paddy,Medak,Kulcharam,82,58,Pea,Lentil,Transplanting,...,11.493762,0.187815,0.282110,3249.392822,2.473036,11,3,4,4,165
2,1326578,Healthy,Paddy,Medak,Kulcharam,92,91,Pea,Lentil,Transplanting,...,10.166884,0.206553,0.309869,3741.956055,1.569470,12,4,4,4,131


### **SPLIT DATA INTO TEST AND TRAIN**

In [18]:
train = data.query('dataset == "train"')

test = data.query('dataset == "test"')

In [19]:
train = train.drop('dataset', axis=1)

test = test.drop(['category', 'dataset'], axis=1)

In [20]:
train.head(3)

,FarmID,category,Crop,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,...,ndwi,gndvi,savi,msavi,AreaHectares,SMonth,SWOM,HMonth,HWOM,SeasonDuration
0,1326576,Healthy,Paddy,Medak,Kulcharam,97,54,Pea,Lentil,Transplanting,...,8.488031,0.127153,0.151125,4232.596191,2.710861,11,4,4,2,141
1,1326577,Healthy,Paddy,Medak,Kulcharam,82,58,Pea,Lentil,Transplanting,...,11.493762,0.187815,0.282110,3249.392822,2.473036,11,3,4,4,165
2,1326578,Healthy,Paddy,Medak,Kulcharam,92,91,Pea,Lentil,Transplanting,...,10.166884,0.206553,0.309869,3741.956055,1.569470,12,4,4,4,131


In [21]:
test.head(3)

,FarmID,Crop,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,...,ndwi,gndvi,savi,msavi,AreaHectares,SMonth,SWOM,HMonth,HWOM,SeasonDuration
8775,85197,Paddy,Medak,Nizampet,81,99,Lentil,Pea,Transplanting,Flood,...,9.219643,0.163744,0.246346,3983.870605,4.791679,11,2,5,3,184
8776,779677,Paddy,Medak,Nizampet,91,63,Lentil,Pea,Transplanting,Flood,...,6.664450,0.052519,0.047902,5052.938477,14.441111,12,4,5,5,160
8777,1331840,Paddy,Medak,Nizampet,94,89,Pea,Lentil,Transplanting,Flood,...,9.922689,0.107397,0.127574,3542.361328,3.230429,11,5,5,3,171


### **FILL MISSING DATA**

In [22]:
train = train.fillna({'ndvi': train['ndvi'].mean(), 
                      'ndwi': train['ndwi'].mean(), 
                      'gndvi': train['gndvi'].mean()})

test = test.fillna({'ndvi': train['ndvi'].mean(), 
                    'ndwi': train['ndwi'].mean(), 
                    'gndvi': train['gndvi'].mean()})

### **REPLACE INFINITY VALUE WITH MEAN**

In [23]:
train['evi'].describe()

c:\Users\Aduragbemi\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


count    7880.000000
mean             inf
std              NaN
min     -1479.291647
25%        -0.966054
50%         0.696779
75%         2.993649
max              inf
Name: evi, dtype: float64

In [24]:
train = train.replace([np.inf, -np.inf], 0.696779)

In [25]:
test['evi'].describe()

c:\Users\Aduragbemi\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


count    2715.000000
mean             inf
std              NaN
min      -237.955184
25%        -1.078173
50%         0.355370
75%         1.944782
max              inf
Name: evi, dtype: float64

In [26]:
test = test.replace([np.inf, -np.inf], 0.355370)

### **SET _FarmID_ AS THE INDEX**

In [27]:
train.set_index('FarmID', inplace=True)

test.set_index('FarmID', inplace=True)

### **CHANGE DATA TYPE OF NON-NUMERIC FEATURE TO _Category_**

In [28]:
train_cat_cols = train.select_dtypes(include='object').columns

for col in train_cat_cols:
    train[col] = train[col].astype('category')

train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7880 entries, 1326576 to 1330504
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   category         7880 non-null   category
 1   Crop             7880 non-null   category
 2   District         7880 non-null   category
 3   Sub-District     7880 non-null   category
 4   CropCoveredArea  7880 non-null   int64   
 5   CHeight          7880 non-null   int64   
 6   CNext            7880 non-null   category
 7   CLast            7880 non-null   category
 8   CTransp          7880 non-null   category
 9   IrriType         7880 non-null   category
 10  IrriSource       7880 non-null   category
 11  IrriCount        7880 non-null   int64   
 12  WaterCov         7880 non-null   int64   
 13  ExpYield         7880 non-null   int64   
 14  Season           7880 non-null   category
 15  ndvi             7880 non-null   float64 
 16  evi              7880 non-null   float

In [29]:
test_cat_cols = test.select_dtypes(include='object').columns

for col in test_cat_cols:
    test[col] = test[col].astype('category')

test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2715 entries, 85197 to 1326552
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Crop             2715 non-null   category
 1   District         2715 non-null   category
 2   Sub-District     2715 non-null   category
 3   CropCoveredArea  2715 non-null   int64   
 4   CHeight          2715 non-null   int64   
 5   CNext            2715 non-null   category
 6   CLast            2715 non-null   category
 7   CTransp          2715 non-null   category
 8   IrriType         2715 non-null   category
 9   IrriSource       2715 non-null   category
 10  IrriCount        2715 non-null   int64   
 11  WaterCov         2715 non-null   int64   
 12  ExpYield         2715 non-null   int64   
 13  Season           2715 non-null   category
 14  ndvi             2715 non-null   float64 
 15  evi              2715 non-null   float64 
 16  ndwi             2715 non-null   float64

### **ENCODE _Categorical_ FEATURES**

In [30]:
for col in train_cat_cols:
    train[col] = train[col].cat.codes

In [31]:
for col in test_cat_cols:
    test[col] = test[col].cat.codes

### **MODEL BUILDING**

In [32]:
X = train.drop(['category'], axis=1)
y = train['category']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### **SCALING**

In [34]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### **SMOTE**

In [35]:
# Oversampling the minority class
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [36]:
model = xgb.XGBClassifier(objective='multi:softmax', 
                          booster='gbtree',
                          num_class=4, 
                          enable_categorical=True, 
                          n_estimators=500,
                          random_state=42)

In [37]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None, num_class=4,
              num_parallel_tree=None, ...)

In [38]:
y_pred = model.predict(X_test)

In [39]:
f1_score(y_test, y_pred, average='weighted')

0.735252308668779

In [40]:
from catboost import CatBoostClassifier

In [41]:
catmodel = CatBoostClassifier(iterations=500, 
                              learning_rate=0.3, 
                              depth=6, 
                              loss_function='MultiClass', 
                              random_state=42, 
                              class_weights=[1, 4, 4, 4])

In [42]:
catmodel.fit(X_train, y_train)

0:	learn: 1.3154368	total: 191ms	remaining: 1m 35s
1:	learn: 1.2295879	total: 217ms	remaining: 54.1s
2:	learn: 1.1825147	total: 270ms	remaining: 44.7s
3:	learn: 1.1482669	total: 313ms	remaining: 38.8s
4:	learn: 1.1166721	total: 343ms	remaining: 33.9s
5:	learn: 1.0925606	total: 386ms	remaining: 31.8s
6:	learn: 1.0569619	total: 424ms	remaining: 29.8s
7:	learn: 1.0312546	total: 467ms	remaining: 28.7s
8:	learn: 1.0098224	total: 499ms	remaining: 27.2s
9:	learn: 0.9900414	total: 524ms	remaining: 25.7s
10:	learn: 0.9752300	total: 547ms	remaining: 24.3s
11:	learn: 0.9512921	total: 569ms	remaining: 23.1s
12:	learn: 0.9413922	total: 592ms	remaining: 22.2s
13:	learn: 0.9265763	total: 622ms	remaining: 21.6s
14:	learn: 0.9095748	total: 664ms	remaining: 21.5s
15:	learn: 0.8952005	total: 707ms	remaining: 21.4s
16:	learn: 0.8898467	total: 741ms	remaining: 21.1s
17:	learn: 0.8785532	total: 772ms	remaining: 20.7s
18:	learn: 0.8643538	total: 802ms	remaining: 20.3s
19:	learn: 0.8571125	total: 838ms	remain

In [43]:
catpred = catmodel.predict(X_test) 

In [44]:
f1_score(y_test, catpred, average='weighted')

0.7367356669729558

In [45]:
import lightgbm as lgb

In [46]:
light_model = lgb.LGBMClassifier(objective='multiclass', 
                                 random_state=42, 
                                 n_estimators=500,
                                 learning_rate=0.3)

In [47]:
light_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002052 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6461
[LightGBM] [Info] Number of data points in the train set: 18240, number of used features: 26
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

LGBMClassifier(learning_rate=0.3, n_estimators=500, objective='multiclass',
               random_state=42)

In [48]:
light_pred = light_model.predict(X_test)

In [49]:
f1_score(y_test, light_pred, average='weighted')

0.7383515797674484

In [50]:
clf = RandomForestClassifier(n_estimators=500, 
                             class_weight='balanced', 
                             random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=500,
                       random_state=42)

In [51]:
clf_pred = clf.predict(X_test)

In [52]:
f1_score(y_test, clf_pred, average='weighted')

0.7382941717250577